<a href="https://colab.research.google.com/github/SeongwonTak/-KDT-_AI_2_Notes/blob/main/Note_Pytorch__Autograd_%26_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch_ Autograd & MLP

In [1]:
import torch

# 연산 과정의 추적 : requires_grad = True
x = torch.ones(2, 2, requires_grad = True)
print(x)

print(x.grad_fn) # 최초 자체 생성으로 None이 된다.

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [2]:
y = x + 2
print(y)
print(y.grad_fn) # 연산의 결과이므로 grad_fn이 생긴다.

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [3]:
z = y * y * 3
out = z.mean()

print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


In [5]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a)
print(a.requires_grad) # 미생성시 False로
a.requires_grad_(True) # True로 바꾸면?
print(a.requires_grad) # 성공
b = (a * a).sum()
print(b)

tensor([[ 0.6491,  1.1257],
        [-4.2669, -2.6000]])
False
True
tensor(26.6551, grad_fn=<SumBackward0>)


## Gradient와 역전파의 진행

In [6]:
y.retain_grad()
z.retain_grad()
out.backward()

In [7]:
print(x.grad)
print(y.grad)
print(z.grad)
print(z.is_leaf) # z가 leaf인가 아닌가?

out.backward()
print(x.grad)
print(y.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
tensor([[0.2500, 0.2500],
        [0.2500, 0.2500]])
False


RuntimeError: ignored

In [8]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()

print(out)

y.retain_grad()
out.backward(retain_graph=True)

print(x.grad)
print(y.grad)
print(z.grad)       # z.retain_grad()를 호출하지 않으면 grad값을 저장하지 않기 때문에 grad 속성을 볼 수 없음
print(z.is_leaf)

out.backward()
print(x.grad)
print(y.grad)

tensor(27., grad_fn=<MeanBackward0>)
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
None
False
tensor([[9., 9.],
        [9., 9.]])
tensor([[9., 9.],
        [9., 9.]])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  del sys.path[0]


In [9]:
x = torch.randn(3, requires_grad=True)
y = x * 2

while y.data.norm() < 1000:
  y = y * 2

print(y)

tensor([ 872.3803,  842.2120, 1169.7627], grad_fn=<MulBackward0>)


In [10]:
# scalar값이 아닌 y의 벡터-야코비안 곱을 구하는 과정
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


# ANN 생성

In [11]:
import pandas as pd

from sklearn.datasets import load_iris

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [12]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.layer0 = nn.Linear(4, 128)
        self.layer1 = nn.Linear(128, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 16)
        self.layer4 = nn.Linear(16, 3)

        self.bn0 = nn.BatchNorm1d(128) #값들을 정규화 한다.
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(32)

        self.act = nn.ReLU()

    def forward(self, x):
      x = self.act(self.bn0(self.layer0(x)))
      x = self.act(self.bn1(self.layer1(x)))
      x = self.act(self.bn2(self.layer2(x)))
      x = self.act(self.layer3(x))
      x = self.layer4(x)

      return x

In [13]:
# 랜덤 값 생성
criterion = nn.CrossEntropyLoss() # 오차

ex_X, ex_y = torch.randn([4, 4]), torch.tensor([1, 0, 2, 0])

net = Net()
output = net(ex_X)
loss = criterion(output, ex_y)
print('loss: ', loss.item())
  
net.zero_grad() # 역전파를 위해 grad 초기화.

print('layer0.bias.grad before backward')
print(net.layer4.bias.grad)

print(net.layer4.bias.is_leaf)

loss.backward()

print('layer0.bias.grad after backward')
print(net.layer4.bias.grad)

loss:  1.1805709600448608
layer0.bias.grad before backward
None
True
layer0.bias.grad after backward
tensor([-0.2529,  0.2101,  0.0428])


In [14]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.001)

optimizer.zero_grad()
output = net(ex_X)
loss = criterion(output, ex_y)
loss.backward()
optimizer.step()  # 업데이트 진행

## Example - Iris.

In [15]:
dataset = load_iris()

data = dataset.data
label = dataset.target

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.25)

In [19]:
# DataLoader 생성
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

train_set = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_set, batch_size=4, shuffle=True)

In [16]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.layer0 = nn.Linear(4, 128)
        self.layer1 = nn.Linear(128, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 16)
        self.layer4 = nn.Linear(16, 3)

        self.bn0 = nn.BatchNorm1d(128)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(32)

        self.act = nn.ReLU()

    def forward(self, x):
      x = self.act(self.bn0(self.layer0(x)))
      x = self.act(self.bn1(self.layer1(x)))
      x = self.act(self.bn2(self.layer2(x)))
      x = self.act(self.layer3(x))
      x = self.layer4(x)

      return x

In [17]:
net = Net()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
epochs = 200

In [ ]:
losses = list()
accuracies = list()

for epoch in range(epochs):
  epoch_loss = 0  
  epoch_accuracy = 0
  for X, y in train_loader:
  
    optimizer.zero_grad()
    
    output = net(X)

    loss = criterion(output, y)
    loss.backward()
    
    optimizer.step()
    # output = [0.11, 0.5, 0.8]  --> 예측 클래스 값
    _, predicted = torch.max(output, dim=1) # 최대값의 index를 뱉는다.
    accuracy = (predicted == y).sum().item()
    epoch_loss += loss.item()
    epoch_accuracy += accuracy
  

  epoch_loss /= len(train_loader)
  epoch_accuracy /= len(X_train)
  print("epoch :{}, \tloss :{}, \taccuracy :{}".format(str(epoch+1).zfill(3),round(epoch_loss,4), round(epoch_accuracy,4)))
  
  losses.append(epoch_loss)
  accuracies.append(epoch_accuracy)

In [21]:
# Test

output = net(X_test)
print(torch.max(output, dim=1))
_, predicted = torch.max(output, dim=1)
accuracy = round((predicted == y_test).sum().item() / len(y_test),4)


print("test_set accuracy :", round(accuracy,4))

torch.return_types.max(
values=tensor([0.8116, 1.8552, 0.8430, 2.6419, 1.2063, 3.0256, 0.9870, 1.1213, 0.9885,
        0.5825, 2.3526, 2.0756, 2.2890, 1.0254, 1.8342, 0.9891, 1.3011, 2.4157,
        1.9383, 2.3550, 2.2660, 2.8542, 1.0786, 0.8992, 1.3460, 1.9769, 2.3171,
        1.2148, 1.8115, 1.6862, 1.3565, 2.6731, 2.6688, 0.9835, 1.6969, 2.3228,
        0.7760, 0.8229], grad_fn=<MaxBackward0>),
indices=tensor([1, 2, 2, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 2, 0, 2, 0, 1, 1,
        2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 1, 1]))
test_set accuracy : 1.0
